In [1]:
import numpy as np
import tensorflow as tf
import PIL as Image
from sklearn.model_selection import train_test_split

In [2]:
PATH = 'D:/GitHub/105_classes_pins_dataset'

In [3]:
data = tf.keras.preprocessing.image_dataset_from_directory(PATH)

Found 17534 files belonging to 105 classes.


In [4]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [7]:
train_generator = datagen.flow_from_directory(PATH,
                                                    batch_size = 1,
                                                    class_mode = 'binary', 
                                                    target_size = (302, 320))    

Found 17534 images belonging to 105 classes.


In [8]:
x=np.concatenate([train_generator.next()[0] for i in range(train_generator.__len__())])
y=np.concatenate([train_generator.next()[1] for i in range(train_generator.__len__())])
print(x.shape)
print(y.shape)

KeyboardInterrupt: 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [ ]:
x_test, x_valid, y_test, y_valid = train_test_split(x_test,y_test)

In [ ]:
x.shape

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
x_valid.shape